In [ ]:
!pip install tkinter
!pip install mediapipe opencv-python pandas scikit-learn

In [2]:
import tkinter as tk
import customtkinter as ck

In [3]:
import mediapipe as mp
import numpy as np
import csv
import os
import pickle
import pandas as pd
import cv2
from matplotlib import pyplot as plt

#To set up the mediapipe
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

#Create array
landmarks = ['class']
for val in range(1, 33+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [7]:
# Load the .pkl file for the first set of data
with open('coords-cleaned.pkl', 'rb') as f:
    model1 = pickle.load(f)

# Load the .pkl file for the second set of data
with open('good-gm-cvin-cvout.pkl', 'rb') as f:
    model2 = pickle.load(f)

feed = ''
counter = 0
current_stage = ''
# List of body landmarks
landmarks = ['class', 'x', 'y', 'z', 'visibility']

def tracker(feed):
    global counter, current_stage, pause
    cap = cv2.VideoCapture(feed)
    global cap
    pause = False
    # initiate holistic model
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        canvas = ck.CTkCanvas(app, width=640, height=480)
        canvas.place(relx=0.5, rely=0.5, anchor='n')

        while cap.isOpened():
            ret, frame = cap.read()

            if not ret:
                break

            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Make Detection
            results = pose.process(image)

            # Recolor image to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # Render detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            try:
                row = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
                X = pd.DataFrame([row], columns=landmarks[1:])
                body_language_class1 = model1.predict(X)[0]
                body_language_prob1 = model1.predict_proba(X)[0]

                body_language_class2 = model2.predict(X)[0]
                body_language_prob2 = model2.predict_proba(X)[0]

                if body_language_class1 == 'up' and body_language_prob1[body_language_prob1.argmax()] >= .7:
                    current_stage = 'up'
                elif current_stage == 'up' and body_language_class1 == 'down' and body_language_prob1[body_language_prob1.argmax()] >= .7:
                    current_stage="down"
                    counter += 1
                    print(current_stage)

                # Get status box
                canvas.create_rectangle(0, 0, 480, 60, fill=(245, 117, 16), outline="")

                # Display Up or Down
                canvas.create_text(95, 12, text='CLASS', font=('Arial', 10), fill='black')
                canvas.create_text(90, 40, text=body_language_class1.split(' ')[0], font=('Arial', 12), fill='white')

                # Display probability
                canvas.create_text(15, 12, text='PROB', font=('Arial', 10), fill='black')
                canvas.create_text(10, 40, text=str(round(body_language_prob1[np.argmax(body_language_prob1)], 2)),
                                   font=('Arial', 12), fill='white')

                # Display counter
                canvas.create_text(180, 12, text='COUNT', font=('Arial', 10), fill='black')
                canvas.create_text(175, 40, text=str(counter), font=('Arial', 12), fill='white')

                # Display Class
                canvas.create_text(260, 12, text='FORM', font=('Arial', 10), fill='black')
                canvas.create_text(265, 40, text=body_language_class2.split(' ')[0], font=('Arial', 12), fill='white')

                # Display probability
                canvas.create_text(385, 12, text='FORM PROB', font=('Arial', 10), fill='black')
                canvas.create_text(390, 40, text=str(round(body_language_prob2[np.argmax(body_language_prob2)], 2)),
                                   font=('Arial', 12), fill='white')

            except Exception as e:
                print("Error")
                pass

            k = cv2.waitKey(1)
            if k == 27:  # press Esc key to exit
                break

            if not pause:
                display_image = cv2.resize(image, (1280, 720))
                canvas.update()
                
            

        cap.release()
        cv2.destroyAllWindows()

        if cv2.waitKey(0) & 0xFF == ord('p'):
            # If 'p' is pressed after a video, pause before proceeding to the next video
            pause = True

SyntaxError: name 'cap' is assigned to before global declaration (460853541.py, line 18)

In [5]:
from tkinter import filedialog

ck.set_appearance_mode("System")
ck.set_default_color_theme("blue")

app = ck.CTk()
app.geometry("1280x720")
app.title("Motion Tracker")

    
def open_video():
    file_path = filedialog.askopenfilename(filetypes=[("Video files", "*.mp4;*.avi;*.mkv")])
    if file_path:
        tracker(file_path)
    
def open_webcam():
    tracker(0)  # 0 represents the default camera (webcam)
    
def close_window():
    cap.release()  # Release the video capture stream
    cv2.destroyAllWindows()

title = ck.CTkLabel(app, text="Choose what to do", font=('Arial',20))
title.place(relx=0.5, rely=0.1, anchor='center')

button1 = ck.CTkButton(app, text="Video", command=open_video, font=('Arial', 20))
button1.place(relx=0.25, rely=0.2, anchor='center')

button2 = ck.CTkButton(app, text="Live", command=open_webcam, font=('Arial', 20))
button2.place(relx=0.75, rely=0.2, anchor='center')

close_button = ck.CTkButton(app, text="Close", command=close_window, font=('Arial', 20))
close_button.place(relx=0.5, rely=0.3, anchor='center')

app.mainloop()


NameError: name 'close_window' is not defined